## Problem Set 2: Merging and regular expressions

**Total points (without extra credit)**: 30 

**Background on the policy context**: here, we're going to use two datasets to practice reshaping, merging, and regular expression patterns. Both datasets relate to the broader issue of which employers might be violating the rights of temporary guestworkers granted visas under the H-2A program. Here are some articles about potential exploitation of guestworkers by firms and inequality caused by minimal oversight:

- News media coverage of labor abuses of temporary guestworkers: https://www.buzzfeednews.com/article/kenbensinger/the-pushovers 
- GAO report on labor abuses of temporary guestworkers: https://www.gao.gov/products/gao-15-154

The following datasets are located in `pset2_inputdata` (need to unzip): 

- `jobs_clean`: a dataset of guestworker jobs posted by many employers, some of whom have been debarred (banned) from the program for labor abuses; others not debarred
- `debar`: a dataset of employers who committed violations of labor regulations meant to protect temporary guestworkers 


You can view a codebook here: https://docs.google.com/spreadsheets/d/1rF9GJEC8pPKxipD0TsoG9DVdqz3EJ-b-BHEtyioAX7I/edit?usp=sharing


In [1]:
## helpful packages
import pandas as pd
import numpy as np
import random
import re
import os

## repeated printouts
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"


# 1. Reshaping data (13 points total)

Load the following dataset stored in `pset2_inputdata`: `debar.csv`

This represents employers temporarily banned from hiring workers (debar.csv); call this `debar`


View the head()


In [2]:
debar_df = pd.read_csv("pset2_inputdata/debar.csv")
debar_df.head()


,Name,"City, State",Violation,Duration,Start date,End date
0,J&J Harvesting,"Leads, ND",Failure to respond to audit (partial response),2 years,1/19/2014,1/18/2016
1,"Stahlman Apiaries, Inc","Selby, SD",Failure to respond to audit (partial response),1 year,2/19/2015,2/14/2016
2,Trust Nursery,"Pulaski, NY",Failure to respond to audit (partial response),1 year,3/21/2014,3/20/2015
3,Anton Fertilizer Inc.,"Dighton, KS",Failure to respond to audit (no response),2 years,3/30/2014,3/29/2016
4,"Great Plains Fluid Service, Inc.","Greensburg, KS",Failure to respond to audit (no response),2 years,3/30/2014,3/29/2016


## 1.1 (1 point)

Print the number of rows in `debar` versus the number of unique employer names (`Name`). Is there one row per employer or multiple rows for some employers?

In [3]:
# number rows in debar 
len(debar_df)

# number of unique employer names 
debar_df["Name"].nunique()

114

98

There are multiple rows for some employers.

## 1.2 Investigating duplicated rows (2 points)

A. Create a new column in `debar`--`is_repeated`-- that tells us whether an employer (`Name`) is repeated > 1 times

*Hint*: there are multiple ways to solve this but some possibilities to get the list of names that are repeated are:
- Using value_counts() on the `Name` variable and extracting the index from that value counts 
- Using groupby to count the rows attached to one name

B. Print the rows where `is_repeated == True` and interpret

C. Subset to the rows where `is_repeated == True` and save that data as `mult_debar`. Print the head() and shape

In [4]:
name_counts = debar_df["Name"].value_counts()
repeated_names = name_counts[name_counts > 1].index
debar_df["is_repeated"] = np.where(debar_df["Name"].isin(repeated_names), True, False) 


In [5]:
debar_df[debar_df["is_repeated"] == True]

,Name,"City, State",Violation,Duration,Start date,End date,is_repeated
6,Annabella Land & Cattle,"Annabella, UT",Non Payment,1 year,5/9/2014,5/9/2015,True
7,Autumn Hill Orchard,"Groton, MA",Failure to respond to audit (no response),2 years,7/6/2014,7/5/2016,True
8,"Caddo Creek Ranch, dba Paradise Ranch","Caddo, TX",Failure to respond to audit (partial response),2 years,7/20/2014,7/19/2016,True
11,Loewen Harvesting LLC,"Brownsville, TX",Failure to respond to audit (partial response),1 year,8/20/2014,8/19/2015,True
12,Rollo Farm Labor Contractor,"Miami, FL",Failure to respond to audit (no response),2 years,8/23/2014,8/22/2016,True
14,Sharon Mathis,"Tifton, GA",Failure to respond to audit (no response),2 years,11/16/2014,11/15/2016,True
15,SRT Farms,"Morton, TX",Failure to respond to audit (no response),2 years,11/16/2014,11/15/2016,True
16,Mark Duncan,"Roosevelt, UT",Failure to respond to audit (no response),2 years,11/16/2014,11/15/2016,True
17,"Maple Ridge Custom Services, LLC","Altheimer, AK",Failure to respond to audit (partial response),2 years,11/16/2014,11/15/2016,True
18,F&W Farms,"Ingalls, KS",Failure to respond to audit (partial response),2 years,12/10/2014,12/9/2016,True


In [6]:
mult_debar = debar_df[debar_df["is_repeated"] == True]
mult_debar.head()
mult_debar.shape
mult_debar

,Name,"City, State",Violation,Duration,Start date,End date,is_repeated
6,Annabella Land & Cattle,"Annabella, UT",Non Payment,1 year,5/9/2014,5/9/2015,True
7,Autumn Hill Orchard,"Groton, MA",Failure to respond to audit (no response),2 years,7/6/2014,7/5/2016,True
8,"Caddo Creek Ranch, dba Paradise Ranch","Caddo, TX",Failure to respond to audit (partial response),2 years,7/20/2014,7/19/2016,True
11,Loewen Harvesting LLC,"Brownsville, TX",Failure to respond to audit (partial response),1 year,8/20/2014,8/19/2015,True
12,Rollo Farm Labor Contractor,"Miami, FL",Failure to respond to audit (no response),2 years,8/23/2014,8/22/2016,True


(32, 7)

,Name,"City, State",Violation,Duration,Start date,End date,is_repeated
6,Annabella Land & Cattle,"Annabella, UT",Non Payment,1 year,5/9/2014,5/9/2015,True
7,Autumn Hill Orchard,"Groton, MA",Failure to respond to audit (no response),2 years,7/6/2014,7/5/2016,True
8,"Caddo Creek Ranch, dba Paradise Ranch","Caddo, TX",Failure to respond to audit (partial response),2 years,7/20/2014,7/19/2016,True
11,Loewen Harvesting LLC,"Brownsville, TX",Failure to respond to audit (partial response),1 year,8/20/2014,8/19/2015,True
12,Rollo Farm Labor Contractor,"Miami, FL",Failure to respond to audit (no response),2 years,8/23/2014,8/22/2016,True
14,Sharon Mathis,"Tifton, GA",Failure to respond to audit (no response),2 years,11/16/2014,11/15/2016,True
15,SRT Farms,"Morton, TX",Failure to respond to audit (no response),2 years,11/16/2014,11/15/2016,True
16,Mark Duncan,"Roosevelt, UT",Failure to respond to audit (no response),2 years,11/16/2014,11/15/2016,True
17,"Maple Ridge Custom Services, LLC","Altheimer, AK",Failure to respond to audit (partial response),2 years,11/16/2014,11/15/2016,True
18,F&W Farms,"Ingalls, KS",Failure to respond to audit (partial response),2 years,12/10/2014,12/9/2016,True


## 1.3 Reshape mult_debar to wide to begin filtering out duplicates (4 points)

You want to separate out two cases:

- Cases where the repeat rows for one employer are due to duplicated data 
- Cases where the repeat rows for one employer represent repeated violations for different issues

There are various ways to check duplicates in this data (eg converting `Violation` to lowercase; replacing spelled-out states with two-dig state codes)

We're going to use the simple rule of:

- A row is a duplicate if, within an employer (defined by Name + City, State), the Start date for each row's violation is the same 

To begin to check this, reshape `mult_debar` to a wide dataframe (`mult_debar_wide`) with the following columns, treating the `Name` and `City, State` as the index for the pivot:

- Name
- City, State
- start_date_viol1
- start_date_viol2

Print the head and shape

In [7]:
# cleaning state names 
state_replacements = {
    "Utah": "UT",
    "Texas": "TX",
    "AK": "AR",
    "Georgia": "GA"
}

# function to replace names with abreviations 
def replace_state(match):
    state_name = match.group(1)
    return state_replacements.get(state_name, "")

# Cleaning state name pattern
state_pattern = r"(Utah|Texas|AK|Georgia)"

# clean names 
mult_debar.loc[:, "City, State"] = mult_debar["City, State"].apply(lambda x: re.sub(state_pattern, replace_state, x))

# switch brownsville to brownsfield 
mult_debar["City, State"] = mult_debar["City, State"].str.replace("Brownfield, TX", "Brownsville, TX")



# make pivot table for duplicates
mult_debar_wide = pd.pivot_table(mult_debar,
                                 values='Start date',
                                 index=["Name", "City, State"],
                                 columns=mult_debar.groupby(["Name", "City, State"]).cumcount() + 1,
                                 aggfunc='first')
mult_debar_wide.reset_index(inplace=True)

mult_debar_wide.columns = ["Name", "City, State", "start_date_viol1", "start_date_viol2"]



/var/folders/sp/m42g974s16vdc06dp0p_mzmw0000gn/T/ipykernel_15659/3677858580.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  mult_debar["City, State"] = mult_debar["City, State"].str.replace("Brownfield, TX", "Brownsville, TX")


In [8]:
mult_debar_wide.head()
mult_debar_wide.shape 
mult_debar_wide

,Name,"City, State",start_date_viol1,start_date_viol2
0,Annabella Land & Cattle,"Annabella, UT",5/9/2014,5/9/2014
1,Autumn Hill Orchard,"Groton, MA",7/6/2014,7/6/2014
2,"Caddo Creek Ranch, dba Paradise Ranch","Caddo, TX",7/20/2014,7/20/2014
3,Cisco Produce Inc.,"Cairo, GA",12/10/2014,12/10/2015
4,Dove Creek Farms,"Mount Vernon, TX",2/9/2018,2/9/2018


(16, 4)

,Name,"City, State",start_date_viol1,start_date_viol2
0,Annabella Land & Cattle,"Annabella, UT",5/9/2014,5/9/2014
1,Autumn Hill Orchard,"Groton, MA",7/6/2014,7/6/2014
2,"Caddo Creek Ranch, dba Paradise Ranch","Caddo, TX",7/20/2014,7/20/2014
3,Cisco Produce Inc.,"Cairo, GA",12/10/2014,12/10/2015
4,Dove Creek Farms,"Mount Vernon, TX",2/9/2018,2/9/2018
5,F&W Farms,"Ingalls, KS",12/10/2014,12/10/2014
6,Loewen Harvesting LLC,"Brownsville, TX",8/20/2014,8/20/2014
7,Macky and Brad Farms,"Plains, TX",2/13/2015,2/13/2015
8,"Maple Ridge Custom Services, LLC","Altheimer, AR",11/16/2014,11/16/2014
9,Mark Duncan,"Roosevelt, UT",11/16/2014,11/16/2014


## 1.4 Filter out duplicates from original debar data (6 points)

A. Using `mult_debar_wide`, add a column `is_dup` that takes value of True for cases where start_date_viol1 == start_date_viol2 marking the row as a duplicate

B. Going back to the original long-format data you loaded at the beginning- `debar`
    - For employers where `is_dup == True` as indicated by your wide-format dataframe, only keep `violnum == viol1`
    - For all other employers (so is_dup == False and ones we didnt need to check duplicates for), keep all violnum
    - Remove the `is_repeated` column from the `debar` data

**Hint**: you can complete part B without a for loop; `pd.concat` with axis = 0 (row binding) is one way

Call the resulting dataframe `debar_clean` and print the shape and # of unique employer names

In [9]:
mult_debar_wide["is_dup"] = np.where(mult_debar_wide.start_date_viol1 == mult_debar_wide.start_date_viol2, True, False)
mult_debar_wide

duplicates = mult_debar_wide[mult_debar_wide['is_dup'] == True]["Name"]

subset_dup = debar_df[debar_df.Name.isin(duplicates)]
subset_not_dup = debar_df[~debar_df.Name.isin(duplicates)]


subset_dup.shape
subset_dup.drop_duplicates("Name", inplace=True)
subset_dup.shape




debar_clean = pd.concat([subset_dup, subset_not_dup], axis=0)

debar_clean

,Name,"City, State",start_date_viol1,start_date_viol2,is_dup
0,Annabella Land & Cattle,"Annabella, UT",5/9/2014,5/9/2014,True
1,Autumn Hill Orchard,"Groton, MA",7/6/2014,7/6/2014,True
2,"Caddo Creek Ranch, dba Paradise Ranch","Caddo, TX",7/20/2014,7/20/2014,True
3,Cisco Produce Inc.,"Cairo, GA",12/10/2014,12/10/2015,False
4,Dove Creek Farms,"Mount Vernon, TX",2/9/2018,2/9/2018,True
5,F&W Farms,"Ingalls, KS",12/10/2014,12/10/2014,True
6,Loewen Harvesting LLC,"Brownsville, TX",8/20/2014,8/20/2014,True
7,Macky and Brad Farms,"Plains, TX",2/13/2015,2/13/2015,True
8,"Maple Ridge Custom Services, LLC","Altheimer, AR",11/16/2014,11/16/2014,True
9,Mark Duncan,"Roosevelt, UT",11/16/2014,11/16/2014,True


(26, 7)

/var/folders/sp/m42g974s16vdc06dp0p_mzmw0000gn/T/ipykernel_15659/3903265484.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  subset_dup.drop_duplicates("Name", inplace=True)


(13, 7)

,Name,"City, State",Violation,Duration,Start date,End date,is_repeated
6,Annabella Land & Cattle,"Annabella, UT",Non Payment,1 year,5/9/2014,5/9/2015,True
7,Autumn Hill Orchard,"Groton, MA",Failure to respond to audit (no response),2 years,7/6/2014,7/5/2016,True
8,"Caddo Creek Ranch, dba Paradise Ranch","Caddo, TX",Failure to respond to audit (partial response),2 years,7/20/2014,7/19/2016,True
11,Loewen Harvesting LLC,"Brownsville, TX",Failure to respond to audit (partial response),1 year,8/20/2014,8/19/2015,True
12,Rollo Farm Labor Contractor,"Miami, FL",Failure to respond to audit (no response),2 years,8/23/2014,8/22/2016,True
...,...,...,...,...,...,...,...
107,Walker Place,"Danville, IL",Failure to comply with the employer's obligati...,2 months,11/19/2019,1/26/2020,False
108,County Fair Farm (company) and Andrew Williams...,"Jefferson, ME",WHD Debarment,3 years,3/8/2017,3/8/2020,False
110,Jesus Ledesma,"Mulberry, FL",Failure to Respond to Audit Request,2 years,2/8/18,2/8/20,False
112,B & R Harvesting and Paul Cruz (individual),"Collins, Georgia",WHD Debarment,3 years,4/9/17,4/9/20,False


# 2. Merging and regex (17 points total)



## 2.1 Load data on job postings

The previous dataset contains a small subset of employers who faced temporary bans due to violations of H-2A program regulations

Since most of the bans have expired, we're going to see which of those employers posted new H-2A jobs in the first quarter of 2021 

Loading the `jobs_clean.csv` data stored in `pset4_inputdata`

In [10]:
jobs_df = pd.read_csv("pset2_inputdata/jobs.csv")


##  2.2 Try inner join on employer name  (2 points)

- Use the `EMPLOYER_NAME` field of the `jobs` dataset
- Use the `Name` field of the `debar_clean` dataset 

A. Use pd.merge with an inner join on those fields to see whether there are any exact matches. 

B. If there are exact matches, print the row(s) with exact matches



In [11]:
exact_matches = pd.merge(jobs_df, debar_clean, left_on="EMPLOYER_NAME", right_on="Name", how="inner")
exact_matches

,CASE_NUMBER,CASE_STATUS,RECEIVED_DATE,DECISION_DATE,TYPE_OF_EMPLOYER_APPLICATION,H2A_LABOR_CONTRACTOR,NATURE_OF_TEMPORARY_NEED,EMERGENCY_FILING,EMPLOYER_NAME,TRADE_NAME_DBA,...,EMAIL_TO_APPLY,WEBSITE_TO_APPLY,TOTAL_ADDENDUM_A_RECORDS,Name,"City, State",Violation,Duration,Start date,End date,is_repeated
0,H-300-20287-876656,Determination Issued - Certification,2020-10-20 09:20:32.010,2020-11-09 00:00:00.000,Individual Employer,Y,Seasonal,Y,Rafael Barajas,NaN,...,NaN,https://seasonaljobs.dol.gov/,7,Rafael Barajas,"Sebring, Florida",Non-payment of certification fee,1 year,9/23/2016,9/22/2017,False


## 2.3 Targeted regex (10 points total)

You want to see if you can increase the exact match rate with some basic cleaning of each 
of the employer name fields in each dataset 

### 2.3.1 Converting to upper (2 points)

A. Convert the `EMPLOYER_NAME` and `Name` fields to uppercase using list comprehension rather than df.varname.str.upper() (it's fine to do a separate list comprehension line for each of the two columns)

B. Print a random sample of 15 values of each result

C. Assign the full vector of uppercase names back to the original data, writing over the original `EMPLOYER_NAME` and `Name` columns 


In [12]:
## insert your code to turn into uppercase here
jobs_df["EMPLOYER_NAME"] = [name.upper() for name in jobs_df["EMPLOYER_NAME"]]
debar_clean["Name"] = [name.upper() for name in debar_clean["Name"]]

In [13]:
## insert your code for the random sample
jobs_df.sample(n=15)
debar_clean.sample(n=15)

,CASE_NUMBER,CASE_STATUS,RECEIVED_DATE,DECISION_DATE,TYPE_OF_EMPLOYER_APPLICATION,H2A_LABOR_CONTRACTOR,NATURE_OF_TEMPORARY_NEED,EMERGENCY_FILING,EMPLOYER_NAME,TRADE_NAME_DBA,...,ADDENDUM_B_HOUSING_ATTACHED,TOTAL_HOUSING_RECORDS,MEALS_PROVIDED,MEALS_CHARGED,MEAL_REIMBURSEMENT_MINIMUM,MEAL_REIMBURSEMENT_MAXIMUM,PHONE_TO_APPLY,EMAIL_TO_APPLY,WEBSITE_TO_APPLY,TOTAL_ADDENDUM_A_RECORDS
2415,H-300-20323-917275,Determination Issued - Certification,2020-11-23 17:13:18.860,2020-12-29 00:00:00.000,Individual Employer,N,Seasonal,N,"LAKE POTATOES, LLP",NaN,...,N,1,N,NaN,12.68,55.0,17013204434,vanrayb@daktel.com,jobsnd.com,0
182,H-300-20248-806927,Determination Issued - Certification,2020-09-18 13:44:14.000,2020-10-09 00:00:00.000,Individual Employer,N,Seasonal,N,"VPC PRODUCE, LLC",NaN,...,N,1,N,NaN,12.68,55.0,12298964802,heatherterry212@gmail.com,http://employgeorgia.com/,0
1352,H-300-20316-908146,Determination Issued - Certification,2020-11-20 13:09:57.290,2020-12-02 00:00:00.000,Individual Employer,N,Seasonal,N,ACADIAN FAMILY FARM,NaN,...,N,1,N,NaN,12.68,55.0,13617722725,NaN,seasonaljobs.dol.gov,0
2341,H-300-20328-924599,Determination Issued - Certification,2020-12-01 13:59:46.950,2020-12-28 00:00:00.000,Individual Employer,N,Seasonal,N,GALEN SCHERESKY FARMS,NaN,...,Y,4,N,NaN,12.68,55.0,17016792578,agservegalen@restel.com,NaN,0
1030,H-300-20296-885744,Determination Issued - Certification,2020-10-28 17:49:40.070,2020-11-20 00:00:00.000,Individual Employer,N,Seasonal,N,D&L FARMS,NaN,...,N,1,N,NaN,12.68,55.0,16202143419,NaN,https://www.kansasworks.com/,0
2015,H-300-20308-896123,Determination Issued - Certification,2020-11-05 11:04:14.650,2020-12-21 00:00:00.000,Association - Agent,N,Seasonal,N,"PLEASANT VIEW GARDENS, INC",NaN,...,N,1,N,NaN,12.68,55.0,16034351719,ronp@pwpvg.com,NaN,1
272,H-300-20269-845826,Determination Issued - Withdrawn,2020-09-29 13:31:45.790,2020-10-14 00:00:00.000,Individual Employer,N,Seasonal,N,RIES FARMS LLC,NaN,...,Y,2,N,NaN,12.68,55.0,16512716325,riesfarms@gmail.com,NaN,1
2071,H-300-20335-931070,Determination Issued - Certification,2020-12-02 10:41:48.370,2020-12-21 00:00:00.000,Individual Employer,N,Seasonal,N,"COOSAW AG, LLC",NaN,...,N,1,Y,12.68,12.68,55.0,18036323873,bradley@coosawfarms.com,NaN,0
739,H-300-20287-876998,Determination Issued - Certification,2020-10-16 09:40:01.750,2020-11-04 00:00:00.000,Individual Employer,Y,Seasonal,N,"F. MADRID HARVESTING, INC.",NaN,...,Y,7,Y,12.68,12.68,55.0,18636732381,f.madridharvesting@gmail.com,NaN,1
2537,H-300-20323-917517,Determination Issued - Certification,2020-11-20 15:21:15.190,2020-12-30 00:00:00.000,Individual Employer,N,Seasonal,N,"ARBOR FIELDS, INC.",NaN,...,Y,2,N,NaN,12.68,55.0,17042832345,sales@arborfields.com,www.ncworks.gov,0


,Name,"City, State",Violation,Duration,Start date,End date,is_repeated
80,M KENESTON CORP,"Rensselaer, NY",Non-payment,2 years,12/31/2015,12/30/2017,False
110,JESUS LEDESMA,"Mulberry, FL",Failure to Respond to Audit Request,2 years,2/8/18,2/8/20,False
64,GONZALO FERNANDEZ*,"Watsonville, California",WHD referral,3 years,8/25/2016,8/24/2019,False
73,XAVIER HORNE,"Lyons, Georgia",Non-payment of certification fee,1 year,6/16/2016,6/15/2017,True
50,E.V. RANCH LLP,"Rifle, CO",Wage Hour Debarment,1 year,11/15/2014,11/14/2015,False
2,TRUST NURSERY,"Pulaski, NY",Failure to respond to audit (partial response),1 year,3/21/2014,3/20/2015,False
99,TRAVIS AND TARA LAMBOURN,"Bowman, North Dakota",Failure to respond to audit request,2 years,12/9/2016,12/9/2018,False
70,JOHN & NETA LEOPKY FARMS,"Seminole, Texas",A violation of the requirements of § 655.135(j...,5 years,7/21/2016,7/20/2018,False
87,LESLIE RENEE DREW,"Lyons, GA",Impeding the Audit Process – Non- Response,2 years,12/17/2015,12/16/2017,False
49,CITY PINESTRAW AND HARVESTING,"Baxley, GA",Impeding the Audit Process – Partial Response;,1 year,4/11/2015,4/1/2016,False


In [87]:
## insert your code for assigning the uppercase names back to the data

### 2.3.2 Cleaning up punctuation (4 points)

You notice that INC, CO, and LLC are sometimes followed by a period (.) but sometimes not

A. For each dataset, write a regex pattern using `re.sub` to remove the . but only if it's preceded by INC, LLC, or CO 

Make sure LLC, INC, CO remain part of the string but just without the dot

B. Test the pattern on the positive and negative example we provide below and print the result. See the Github issue for examples of what to return


**Hint**: https://stackoverflow.com/questions/7191209/python-re-sub-replace-with-matched-content



In [14]:
pos_example_1 = "CISCO PRODUCE INC."
pos_example_2 = "AVOYELLES HONEY CO., LLC"
neg_example = "E.V. RANCH LLP"

In [15]:
## insert your code here with the regex pattern for part A
pattern = r"(INC|LLC|CO)\."

## insert your code to use re.sub to apply the pattern to the test cases for part B
re.sub(pattern, r"\1", pos_example_1)
re.sub(pattern, r"\1", pos_example_2)
re.sub(pattern, r"\1", neg_example)

'CISCO PRODUCE INC'

'AVOYELLES HONEY CO, LLC'

'E.V. RANCH LLP'

### 2.3.3 (4 points)

Use that pattern in conjunction with `re.sub` and list comprehension to clean the employer name columns in each dataset. Save the new columns as `name_clean` in each. Then, use row subsetting to (1) subset to rows that changed names and (2) for:

- `debar_clean` print the `Name` and `name_clean` columns
- `jobs` print the `EMPLOYER_NAME` and `name_clean` columns

Make sure to use the uppercase versions of the variables


In [16]:
## your code here to clean the columns
debar_clean["name_clean"] = debar_clean["Name"].apply(lambda name: re.sub(pattern, r"\1", name))
jobs_df["name_clean"] = jobs_df["EMPLOYER_NAME"].apply(lambda name: re.sub(pattern, r"\1", name))

changed_names_debar = debar_clean[debar_clean['Name'] != debar_clean['name_clean']]
changed_names_jobs = jobs_df[jobs_df['EMPLOYER_NAME'] != jobs_df['name_clean']]


In [17]:
## your code here to print the head
changed_names_debar[["Name", "name_clean"]].head()
changed_names_jobs[["EMPLOYER_NAME", "name_clean"]].head()

,Name,name_clean
3,ANTON FERTILIZER INC.,ANTON FERTILIZER INC
4,"GREAT PLAINS FLUID SERVICE, INC.","GREAT PLAINS FLUID SERVICE, INC"
5,PROMAX INC.,PROMAX INC
13,REIMER'S INC.,REIMER'S INC
19,CISCO PRODUCE INC.,CISCO PRODUCE INC


,EMPLOYER_NAME,name_clean
4,"DUNSON HARVESTING, INC.","DUNSON HARVESTING, INC"
7,"FARM LABOR ASSOCIATION FOR GROWERS, INC.","FARM LABOR ASSOCIATION FOR GROWERS, INC"
14,"MCLAIN FARMS, INC.","MCLAIN FARMS, INC"
17,"BONNIE PLANTS, INC.","BONNIE PLANTS, INC"
18,"B & W QUALITY GROWERS, INC.","B & W QUALITY GROWERS, INC"


## 2.4 More joins and more cleaning (5 points)

A. Conduct another inner join between `jobs` and `debar_clean` now using the `name_clean` column; print the result. Did the cleaning result in any more employers matched between the two datasets?

B. Create a new column in `debar_clean` called `name_clean_2` that uses regex to take the following name in that dataset:

- `SLASH E.V. RANCH LLP` in the `debar_clean` dataset

And cleans it up so that it matches with this employer in `jobs`

- `SLASH EV RANCH` in the `jobs` dataset

Eg a pattern to remove the dots in the EV and the space+LLP-- you can apply the pattern to all employer names in debar_clean (so don't need to worry about only applying it to that one employer)


C. Conduct a left join using `name_clean_2` as the join column where the left hand dataframe is `jobs`; right hand dataframe is `debar_clean`, store the result as a dataframe, and print the rows where the merge indicator indicates the row was found in both dataframe

**Note**: this manual cleaning process is inefficient and helps motivate why talked about fuzzy matching. Fuzzy matching could recognize that Slash EV ranch is a highly similar string to slash ev ranch llp and match them without us needing to use regex to make the strings identical.

In [18]:
## your code here

jobs_debar = pd.merge(jobs_df, debar_clean, on="name_clean", how="inner")
jobs_debar



,CASE_NUMBER,CASE_STATUS,RECEIVED_DATE,DECISION_DATE,TYPE_OF_EMPLOYER_APPLICATION,H2A_LABOR_CONTRACTOR,NATURE_OF_TEMPORARY_NEED,EMERGENCY_FILING,EMPLOYER_NAME,TRADE_NAME_DBA,...,WEBSITE_TO_APPLY,TOTAL_ADDENDUM_A_RECORDS,name_clean,Name,"City, State",Violation,Duration,Start date,End date,is_repeated
0,H-300-20287-876656,Determination Issued - Certification,2020-10-20 09:20:32.010,2020-11-09 00:00:00.000,Individual Employer,Y,Seasonal,Y,RAFAEL BARAJAS,NaN,...,https://seasonaljobs.dol.gov/,7,RAFAEL BARAJAS,RAFAEL BARAJAS,"Sebring, Florida",Non-payment of certification fee,1 year,9/23/2016,9/22/2017,False


In [19]:
regex_pattern = r"(\.)|(\sLLP)"

re.sub(regex_pattern, r"", "SLASH E.V. RANCH LLP") 

debar_clean["name_clean_2"] = debar_clean["name_clean"].apply(lambda name: re.sub(regex_pattern, r"\1", name))


final_joined = pd.merge(jobs_df, debar_clean, left_on="name_clean", right_on="name_clean_2", how="left", indicator=True)
print(final_joined[final_joined["_merge"]== "both"].shape[0])

'SLASH EV RANCH'

1


# 3. Optional extra credit 1: regex to separate companies from individuals (1 point)

You notice some employers in `debar_clean` have both the name of the company and the name of individual, e.g.:
    
COUNTY FAIR FARM (COMPANY) AND ANDREW WILLIAMSON (INDIVIDUAL)*

Use the uppercase/cleaned `name_clean` in `debar_clean`

A. Write a regex pattern that does the following:
    - Captures the pattern that occurs before COMPANY if (COMPANY) is in string; so in example above, extracts COUNTY FAIR FARM 
    - Captures the pattern that occurs before INDIVIDUAL if (INDIVIDUAL) is also in string -- so in above, extracts ANDREW WILLIAMSON (so omit the "and")
    
B. Test the pattern on `pos_example` and `neg_example`-- make sure former returns a list (if using find.all) or match object (if using re.search) with the company name and individual name separated out; make sure latter returns empty
    
**Hints and resources**: for step A, you can either use re.search, re.match, or re.findall; don't worry about matching B&R Harvesting and Paul Cruz (Individual)

- Same regex resources as above
    

In [20]:
# Regex pattern
pattern = r'^(.*?) \(COMPANY\)(?: AND (.*?))? \(INDIVIDUAL\)*'

# Test strings
pos_example = "COUNTY FAIR FARM (COMPANY) AND ANDREW WILLIAMSON (INDIVIDUAL)"
neg_example = "JUST A REGULAR STRING WITHOUT TAGS"

# Using findall to test the pattern
pos_test = re.findall(pattern, pos_example)
neg_test = re.findall(pattern, neg_example)

print("Positive Test Result:", pos_test)
print("Negative Test Result:", neg_test)

Positive Test Result: [('COUNTY FAIR FARM', 'ANDREW WILLIAMSON')]
Negative Test Result: []


C. Iterate over the `name_clean` column in debar and use regex to create two new columns in `debar_clean`:
   - `co_name`: A column for company (full `name_clean` string if no match; pattern before COMPANY if one extracted)
   - `ind_name`: A column for individual (full `name_clean` string if no match; pattern before INDIVIDUAL if one extracted)
 


In [21]:
#your code here
def split_names(row):
    match = re.search(pattern, row['name_clean'])
    if match:
        # Capture group 1 is company, group 2 (if present) is individual
        return match.group(1).strip(), match.group(2).strip() if match.group(2) else ""
    else:
        # If no match, assume entire string could be a company name
        return row['name_clean'], ""

# Applying the function to the DataFrame
debar_clean[['co_name', 'ind_name']] = debar_clean.apply(split_names, axis=1, result_type='expand')

# Showing the updated DataFrame
print(debar_clean)


                                                  Name       City, State  \
6                              ANNABELLA LAND & CATTLE     Annabella, UT   
7                                  AUTUMN HILL ORCHARD        Groton, MA   
8                CADDO CREEK RANCH, DBA PARADISE RANCH         Caddo, TX   
11                               LOEWEN HARVESTING LLC   Brownsville, TX   
12                         ROLLO FARM LABOR CONTRACTOR         Miami, FL   
..                                                 ...               ...   
107                                       WALKER PLACE      Danville, IL   
108  COUNTY FAIR FARM (COMPANY) AND ANDREW WILLIAMS...     Jefferson, ME   
110                                      JESUS LEDESMA      Mulberry, FL   
112        B & R HARVESTING AND PAUL CRUZ (INDIVIDUAL)  Collins, Georgia   
113                                        DELIA ROJAS    Lyons, Georgia   

                                             Violation  Duration  Start date  \
6      

   
D. Print three columns for the rows in `debar_clean` containing the negative example and positive example described above (county fair farm and cisco produce):

- `name_clean`
- `co_name`
- `ind_name`
- `Violation`

**Note**: as shown in the outcome there may be duplicates of the same company reflecting different violations

In [22]:
# Filtering the DataFrame
filtered_df = debar_clean[(debar_clean['name_clean'].str.contains("COUNTY FAIR FARM")) | (debar_clean['name_clean'].str.contains("CISCO PRODUCE"))]

# Selecting specific columns to display
print(filtered_df[['name_clean', 'co_name', 'ind_name', 'Violation']])


                                            name_clean            co_name  \
19                                   CISCO PRODUCE INC  CISCO PRODUCE INC   
56                                   CISCO PRODUCE INC  CISCO PRODUCE INC   
108  COUNTY FAIR FARM (COMPANY) AND ANDREW WILLIAMS...   COUNTY FAIR FARM   

              ind_name                                   Violation  
19                       Failure to respond to audit (no response)  
56                      Impeding the Audit Process – Non- Response  
108  ANDREW WILLIAMSON                               WHD Debarment  


# 4. Optional extra credit 2 (up to 3 points)

- For 1 point extra credit, create a visualization with 1+ of the existing fields in either the raw `jobs` or `debar` data. We'll be showing cool visualizations in class so use your imagination! Options could include visualizing between-state or over-time variation

- For 3 points extra credit instead, geocode the employer addresses in `jobs` and plot the addresses of jobs as points overlaid on top of a map of Georgia 
    - **Note**: this extra credit involves Googling since we have not yet covered spatial data. 
        - For discussion of how to geocode addresses -> lat/long, see: https://www.natasshaselvaraj.com/a-step-by-step-guide-on-geocoding-in-python/ 
        - For discussion of plotting lat/long dots against a map, see this discussion of geopandas: https://towardsdatascience.com/plotting-maps-with-geopandas-428c97295a73
    - Relevant columns include `EMPLOYER_ADDRESS_1` 
    - The geocoding might have a long runtime so feel free to implement it in a separate .py script that you submit alongside your notebook and to just read in the geocoded data

In [23]:

## THIS CODE IS AN INCOMPLETE ATTEMPT AT PARTIAL CREDIT 
import geopandas as gpd
import urllib
import requests
import googlemaps

jobs_df = pd.read_csv("pset2_inputdata/jobs.csv")


jobs_location = jobs_df[["EMPLOYER_ADDRESS_1"]]
jobs_location

url = 'https://nominatim.openstreetmap.org/search' + urllib.parse.quote(jobs_df['EMPLOYER_ADDRESS_1'][0]) +'?format=json'

print(url)
response = requests.get(url).json()

url = 'http://localhost/nominatim/'
url = 'localhost'
g = geocoder.osm(jobs_df['EMPLOYER_ADDRESS_1'][0], url=url)
g.json
# print('Latitude: '+response[0]['lat']+', Longitude: '+response[0]['lon'])

def geocode2(locality):
    # Construct the URL for geocoding
    url = 'https://nominatim.openstreetmap.org/search/' + urllib.parse.quote(locality) + '?format=json'
    
    # Send a GET request to the Nominatim API
    response = requests.get(url).json()
    
    # Check if the response contains any results
    if len(response) > 0:
        # Extract latitude and longitude from the first result
        latitude = response[0]['lat']
        longitude = response[0]['lon']
        return latitude, longitude
    else:
        # Return None if no results found
        return None

jobs_location['geocoded'] = jobs_location['EMPLOYER_ADDRESS_1'].apply(geocode2)

# Save the DataFrame
geocoded_jobs_df.to_csv("pset2_inputdata/geocoded_jobs.csv", index=False)

# Load geocoded data
geocoded_jobs = pd.read_csv("pset2_inputdata/geocoded_jobs.csv")

# Convert DataFrame to GeoDataFrame
gdf = gpd.GeoDataFrame(
    geocoded_jobs, geometry=gpd.points_from_xy(geocoded_jobs.lon, geocoded_jobs.lat))

# Load a map of Georgia
world = gpd.read_file(gpd.datasets.get_path('naturalearth_lowres'))
georgia_map = world[world.name == "Georgia"]

# # Plot
# fig, ax = plt.subplots(figsize=(10, 10))
# georgia_map.plot(ax=ax, color='lightgrey')
# gdf.plot(ax=ax, markersize=10, color='red', marker='o', label='Job Locations')
# plt.title('Job Locations in Georgia')
# plt.legend()
# plt.show()

,EMPLOYER_ADDRESS_1
0,8433 NE 13th Ave
1,982 Northcutts Cove Road
2,7366 State Hwy 146 S
3,4198 Road 2049
4,400 Eagle Lake Loop Rd.
...,...
2715,1448 Little Knob Road
2716,5850 W Jeff Davis Hwy
2717,2412 Povey Road
2718,231 Tendoy Street - Note: No Mail Delivery


https://nominatim.openstreetmap.org/search8433%20NE%2013th%20Ave?format=json


NameError: name 'geocoder' is not defined

In [80]:
import geocoder
url = 'http://localhost/nominatim/'
url = 'localhost'
g = geocoder.osm(jobs_df['EMPLOYER_ADDRESS_1'][0])

def geocode(locality): 
    try:
        # Use geocoder to retrieve the geolocation data
        g = geocoder.osm(locality, timeout=10)  # Increase the timeout to 10 seconds
        if g.ok:
            return g.latlng  # Return latitude and longitude as a tuple
        else:
            return None  # Return None if geocoding fails
    except Exception as e:
        print("Error:", e)
        return None  # Return None if an error occurs during geocoding

jobs_location['geocoded'] = jobs_location['EMPLOYER_ADDRESS_1'].apply(geocode)


Status code Unknown from https://nominatim.openstreetmap.org/search: ERROR - HTTPSConnectionPool(host='nominatim.openstreetmap.org', port=443): Max retries exceeded with url: /search?q=8433+NE+13th+Ave&format=jsonv2&addressdetails=1&limit=1 (Caused by NameResolutionError("<urllib3.connection.HTTPSConnection object at 0x149cadb50>: Failed to resolve 'nominatim.openstreetmap.org' ([Errno 8] nodename nor servname provided, or not known)"))
Status code Unknown from https://nominatim.openstreetmap.org/search: ERROR - HTTPSConnectionPool(host='nominatim.openstreetmap.org', port=443): Max retries exceeded with url: /search?q=8433+NE+13th+Ave&format=jsonv2&addressdetails=1&limit=1 (Caused by NameResolutionError("<urllib3.connection.HTTPSConnection object at 0x149c25150>: Failed to resolve 'nominatim.openstreetmap.org' ([Errno 8] nodename nor servname provided, or not known)"))
Status code Unknown from https://nominatim.openstreetmap.org/search: ERROR - HTTPSConnectionPool(host='nominatim.opens